### Crawl Bus Final


In [2]:
import requests
import json
import time

# --- CẤU HÌNH ---
OUTPUT_FILE = "../data/hcm_bus_final.json"
MAX_ID_SCAN = 407 # Quét dư ra cho chắc (thường chỉ đến 150-200)

# CÁC API
API_DETAIL = "http://apicms.ebms.vn/businfo/getroutebyid/{}"
API_VARS   = "http://apicms.ebms.vn/businfo/getvarsbyroute/{}"
API_STOPS  = "http://apicms.ebms.vn/businfo/getstopsbyvar/{}/{}"

def crawl_universal():
    print(f"🚀 BẮT ĐẦU QUÉT TOÀN BỘ HỆ THỐNG (ID 1 -> {MAX_ID_SCAN})...")
    print("ℹ️ Cơ chế: Tự động phát hiện mã Var.")
    
    final_routes = []
    unique_stops_map = {} 
    
    count_success = 0

    for r_id in range(1, MAX_ID_SCAN + 1):
        try:
            # 1. LẤY THÔNG TIN CƠ BẢN
            resp = requests.get(API_DETAIL.format(r_id), timeout=2)
            info = resp.json()
            
            # Nếu ID rác -> Bỏ qua
            if not info or 'RouteId' not in info: continue
            
            r_no = info['RouteNo']
            
            # 2. [QUAN TRỌNG] TỰ ĐỘNG DÒ TÌM MÃ VAR
            # Hỏi server xem tuyến này có những chiều nào (1, 2, 13, 14, 100...?)
            target_vars = []
            try:
                var_resp = requests.get(API_VARS.format(r_id), timeout=2)
                var_data = var_resp.json()
                # Trích xuất tất cả RouteVarId
                target_vars = [v['RouteVarId'] for v in var_data]
            except:
                pass
            
            # Fallback: Nếu API Var lỗi, mới dùng mặc định 1, 2 (nhưng thường API Var rất chuẩn)
            if not target_vars: 
                target_vars = [1, 2]
                print(f"   ⚠️ Bus {r_no} không lấy được Var, dùng mặc định [1, 2]")

            # 3. TẠO OBJECT TUYẾN
            route_obj = {
                "RouteId": info.get('RouteId'),
                "RouteNo": info.get('RouteNo'),
                "RouteName": info.get('RouteName'),
                "Distance": info.get('Distance'),
                "TimeOfTrip": info.get('TimeOfTrip'),
                "OperationTime": info.get('OperationTime'),
                "Tickets": info.get('Tickets'),
                "Orgs": info.get('Orgs'),
                "Stops": [] 
            }

            has_stops = False
            
            # 4. DUYỆT QUA CÁC VAR ĐÃ TÌM THẤY (Dynamic Loop)
            for var in target_vars:
                try:
                    s_resp = requests.get(API_STOPS.format(r_id, var), timeout=3)
                    stops_list = s_resp.json()
                    
                    if stops_list: has_stops = True
                    
                    for s in stops_list:
                        stop_id = s['StopId']
                        
                        # A. Lưu Trạm (Unique)
                        if stop_id not in unique_stops_map:
                            unique_stops_map[stop_id] = {
                                "StopId": s['StopId'],
                                "Code": s['Code'],
                                "Name": s['Name'],
                                "Lat": s['Lat'],
                                "Lng": s['Lng'],
                                "Street": s['Street'],
                                "Zone": s['Zone'],
                                "Address": s['AddressNo'],
                                "Search": s['Search'],
                                "Routes": s['Routes']
                            }
                        
                        # B. Gán ID Trạm vào Tuyến
                        route_obj["Stops"].append({
                            "StopId": stop_id,
                            "Direction": var, # Lưu đúng mã Var thực tế (vd: 13)
                            "Order": len(route_obj["Stops"]) + 1
                        })
                except:
                    pass

            # Chỉ lưu nếu lấy được dữ liệu trạm
            if has_stops:
                final_routes.append(route_obj)
                count_success += 1
                # In ra để bạn kiểm tra: Thấy Bus 07 hiện Var [13, 14] là Uy tín!
                print(f"   ✅ Bus {r_no} (ID {r_id}) -> Vars: {target_vars} | {len(route_obj['Stops'])} trạm")
            
        except Exception as e:
            print(f"   ❌ Lỗi quét ID {r_id}: {e}")
            
        # Nghỉ xíu
        # time.sleep(0.01)

    # 5. LƯU FILE
    master_data = {
        "Routes": final_routes,
        "UniqueStops": list(unique_stops_map.values())
    }

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(master_data, f, ensure_ascii=False, indent=2)
        
    print(f"\n🎉 HOÀN TẤT! Đã cào {count_success} tuyến xe buýt.")
    print(f"📂 File kết quả: {OUTPUT_FILE}")

if __name__ == "__main__":
    crawl_universal()

🚀 BẮT ĐẦU QUÉT TOÀN BỘ HỆ THỐNG (ID 1 -> 407)...
ℹ️ Cơ chế: Tự động phát hiện mã Var.
   ✅ Bus 01 (ID 1) -> Vars: [1, 2] | 61 trạm
   ✅ Bus 03 (ID 3) -> Vars: [5, 6] | 107 trạm
   ✅ Bus 04 (ID 4) -> Vars: [7, 8] | 85 trạm
   ✅ Bus 05 (ID 5) -> Vars: [2, 3] | 124 trạm
   ✅ Bus 06 (ID 6) -> Vars: [1, 2] | 115 trạm
   ✅ Bus 07 (ID 7) -> Vars: [13, 14] | 97 trạm
   ✅ Bus 08 (ID 8) -> Vars: [1, 2] | 145 trạm
   ✅ Bus 09 (ID 9) -> Vars: [1, 2] | 119 trạm
   ✅ Bus 10 (ID 10) -> Vars: [1, 2] | 143 trạm
   ✅ Bus 100 (ID 11) -> Vars: [1, 2] | 67 trạm
   ✅ Bus 101 (ID 12) -> Vars: [1, 2] | 74 trạm
   ✅ Bus 102 (ID 13) -> Vars: [25, 26] | 120 trạm
   ✅ Bus 103 (ID 14) -> Vars: [1, 2] | 126 trạm
   ✅ Bus 104 (ID 15) -> Vars: [1, 2] | 80 trạm
   ✅ Bus 107 (ID 16) -> Vars: [31, 32] | 60 trạm
   ✅ Bus 110 (ID 18) -> Vars: [1, 2] | 102 trạm
   ✅ Bus 122 (ID 20) -> Vars: [39, 40] | 81 trạm
   ✅ Bus 126 (ID 23) -> Vars: [45, 46] | 78 trạm
   ✅ Bus 13 (ID 24) -> Vars: [47, 48] | 155 trạm
   ✅ Bus 139 (ID 